In [9]:
import  numpy as np

In [10]:
def Sigmoide(x):
    return 1/(1+np.exp(-x))


def my_dense(a_in, W, b):
    units = W.shape[1]
    a_out = np.zeros(units)
    for j in range(units):
        w = W[:,j]
        z = np.dot(w,a_in)+ b[j]
        a_out[j] = Sigmoide(z)
    return a_out

def my_Sequential(x, W1, b1, W2, b2):
    a1 = my_dense(x, W1, b1)
    a2 = my_dense(a1, W2, b2)
    return a2

def my_predict(X, W1, b1, W2, b2):
    N = X.shape[0]
    y = np.zeros((N, 1))  # Initialize y as a 2D array
    for i in range(N):
        y[i] = my_Sequential(X[i], W1, b1, W2, b2)
    return y

Now let's test our functions

In [11]:
W1_tmp = np.array( [[-8.93,  0.29, 12.9 ], [-0.1,  -7.32, 10.81]] )
b1_tmp = np.array( [-9.82, -9.28,  0.96] )
W2_tmp = np.array( [[-31.18], [-27.59], [-32.56]] )
b2_tmp = np.array( [15.41] )

In [12]:
import tensorflow as tf

# Create the normalization layer
norm_l = tf.keras.layers.Normalization(axis=-1)

# Adapt the normalizer to the given ranges
data = np.array([
    [284.99, 15.45],
    [151.32, 11.51]
])
norm_l.adapt(data)

In [13]:
X_tst = np.array([
    [200,13.9],  
    [200,17]])   
X_tstn = norm_l(X_tst)  # remember to normalize
predictions = my_predict(X_tstn, W1_tmp, b1_tmp, W2_tmp, b2_tmp)

In [14]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]


In [15]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1 
    else:       
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]
